In [1]:
!pip3 install pyvi keras gensim
!pip3 install --upgrade keras tensorflow
!pip3 install "tensorflow-text==2.13.*"

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-3.0.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.7-py3-none-any.whl (5.8 kB)
  Using cached dm_tree-0.1.8-cp39-cp39-macosx_11_0_arm64.whl (110 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.13.* (from versions: none)
ERROR: No matching distribution found for tensorflow-text==2.13.*


In [53]:
from __future__ import division, print_function
from gensim import models
# from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
# from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import re
from string import digits
from collections import Counter
from pyvi import ViTokenizer

In [54]:
data_train = pd.read_csv("./vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv("./vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [55]:
print(data_train.shape)
print(data_test.shape)

(5100, 2)
(1050, 2)


In [56]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

In [57]:
with open("./stopwords.txt", "r", encoding="utf8") as f:
    stopwords = f.read().splitlines()

def remove_stop_words(text, stopwords):
    words = text.split()
    words = [word for word in words if word not in stopwords]
    text = ' '.join(words)
    return text

In [58]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)  

In [59]:
reviews_processed = []
unlabeled_processed = [] 
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    # review_cool_one = remove_stop_words(review_cool_one, stopwords)
    reviews_processed.append(review_cool_one)

In [60]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())

In [61]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [62]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [63]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index

In [70]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [71]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (5100, 300)
Shape of label train and validation tensor: (5100, 3)


In [66]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format("./vi-model-CBOW.bin", binary=True)


vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [30]:
!pip3 install keras-self-attention

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=b3fc6640b379d5e892bd8bdb77dcf8d6887932faa1c4de606047e5da47e018bf
  Stored in directory: /Users/thunguyen/Library/Caches/pip/wheels/78/c1/84/b83a2fd6f1d63e136cba74bac4126bee3b8705eef6486635fd
Successfully built keras-self-attention


In [80]:
from keras.models import Model
from keras.layers import Input, Embedding, Reshape, Conv1D, MaxPooling1D, Dropout, LSTM, Dense, concatenate, Attention, Concatenate
from keras.optimizers import Adam
from keras import regularizers

sequence_length = data.shape[1]
filter_sizes = [3, 4, 5]
num_filters = 100
drop = 0.5

input = Input(shape=(sequence_length,))
embedding = embedding_layer(input)
reshape = Reshape((sequence_length, EMBEDDING_DIM))(embedding)

conv_0 = Conv1D(num_filters, filter_sizes[0], activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
maxpool_0 = MaxPooling1D(sequence_length-filter_sizes[0]+1, strides=1)(conv_0)

conv_1 = Conv1D(num_filters, filter_sizes[1], activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
maxpool_1 = MaxPooling1D(sequence_length-filter_sizes[1]+1, strides=1)(conv_1)

conv_2 = Conv1D(num_filters, filter_sizes[2], activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
maxpool_2 = MaxPooling1D(sequence_length-filter_sizes[2]+1, strides=1)(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2])
# merged_tensor = Reshape((-1, 1))(merged_tensor) 
lstm_2 = LSTM(units=128, return_sequences=False)(merged_tensor)

# attention = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                            #  kernel_regularizer=regularizers.l2(0.01))(lstm_2)

attention = Attention()([lstm_2, lstm_2])

attended_output = Concatenate(axis=-1)([lstm_2, attention])

dropout = Dropout(rate=drop)(attention)
output = Dense(units=3, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(dropout)

model = Model(inputs=input, outputs=output)
model.compile(loss='categorical_crossentropy',
            optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
            metrics=['accuracy'])
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_21 (InputLayer)       [(None, 300)]                0         []                            
                                                                                                  
 embedding_14 (Embedding)    (None, 300, 400)             3167600   ['input_21[0][0]']            
                                                                                                  
 conv1d_52 (Conv1D)          (None, 298, 100)             120100    ['embedding_14[5][0]']        
                                                                                                  
 conv1d_53 (Conv1D)          (None, 297, 100)             160100    ['embedding_14[5][0]']        
                                                                                           

In [81]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

np.random.seed(42)

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, shuffle=True, random_state=42)

model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=256, callbacks=callbacks_list)
# model.save_weights("./model.h5")

Epoch 1/10
16/16 [==============================] - 15s 867ms/step - loss: 4.6465 - accuracy: 0.3350 - val_loss: 3.2397 - val_accuracy: 0.3510
Epoch 2/10
16/16 [==============================] - 13s 811ms/step - loss: 2.5030 - accuracy: 0.3311 - val_loss: 1.8504 - val_accuracy: 0.4049
Epoch 3/10
16/16 [==============================] - 13s 834ms/step - loss: 1.5492 - accuracy: 0.4020 - val_loss: 1.2881 - val_accuracy: 0.4725
Epoch 4/10
16/16 [==============================] - 14s 884ms/step - loss: 1.1967 - accuracy: 0.4961 - val_loss: 1.1411 - val_accuracy: 0.4784
Epoch 5/10
16/16 [==============================] - 14s 862ms/step - loss: 1.0963 - accuracy: 0.5203 - val_loss: 1.1365 - val_accuracy: 0.4559
Epoch 6/10
16/16 [==============================] - 15s 935ms/step - loss: 1.0238 - accuracy: 0.5395 - val_loss: 1.1039 - val_accuracy: 0.4941
Epoch 7/10
16/16 [==============================] - 14s 892ms/step - loss: 0.9714 - accuracy: 0.5534 - val_loss: 1.1253 - val_accuracy: 0.4951

In [82]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [83]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)  

In [84]:
reviews_processed_test = []
unlabeled_processed_test = [] 
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    # review_cool_one = remove_stop_words(review_cool_one, stopwords)
    reviews_processed_test.append(review_cool_one)

In [85]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [86]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [87]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


In [88]:
score = model.evaluate(data_test, labels_test)

33/33 [==============================] - 1s 30ms/step - loss: 0.9744 - accuracy: 0.6971


In [89]:
print("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

loss: 97.44%
accuracy: 69.71%
